In [1]:
from sqlalchemy import create_engine, ForeignKey, Column, Integer, String
engine = create_engine('sqlite:///mycollege.db', echo = True)
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import relationship

In [2]:
class Department(Base):
    __tablename__ = 'department'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    employees = relationship('Employee', secondary = 'link')
    
class Employee(Base):
    __tablename__ = 'employee'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    departments = relationship(Department,secondary='link')

In [3]:
class Link(Base):
    __tablename__ = 'link'
    department_id = Column(
        Integer, 
        ForeignKey('department.id'), 
        primary_key = True)

    employee_id = Column(
       Integer, 
       ForeignKey('employee.id'), 
       primary_key = True)

In [4]:
Base.metadata.create_all(engine)

2020-12-04 12:14:43,320 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-04 12:14:43,320 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:14:43,322 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-04 12:14:43,323 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:14:43,323 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("department")
2020-12-04 12:14:43,324 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:14:43,325 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("department")
2020-12-04 12:14:43,325 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:14:43,326 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("employee")
2020-12-04 12:14:43,326 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:14:43,326 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("employee")
2020-12-04 12:14:43,327 INFO sqlalchemy.engine.base.Engine ()
2020-12-04

In [5]:
d1 = Department(name = "Accounts")
d2 = Department(name = "Sales")
d3 = Department(name = "Marketing")

e1 = Employee(name = "John")
e2 = Employee(name = "Tony")
e3 = Employee(name = "Graham")

In [6]:
e1.departments.append(d1)
e2.departments.append(d3)
d1.employees.append(e3)
d2.employees.append(e2)
d3.employees.append(e1)
e3.departments.append(d2)

In [7]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()
session.add(e1)
session.add(e2)
session.add(d1)
session.add(d2)
session.add(d3)
session.add(e3)
session.commit()

2020-12-04 12:14:43,365 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-04 12:14:43,367 INFO sqlalchemy.engine.base.Engine INSERT INTO department (name) VALUES (?)
2020-12-04 12:14:43,368 INFO sqlalchemy.engine.base.Engine ('Accounts',)
2020-12-04 12:14:43,369 INFO sqlalchemy.engine.base.Engine INSERT INTO department (name) VALUES (?)
2020-12-04 12:14:43,369 INFO sqlalchemy.engine.base.Engine ('Sales',)
2020-12-04 12:14:43,370 INFO sqlalchemy.engine.base.Engine INSERT INTO department (name) VALUES (?)
2020-12-04 12:14:43,370 INFO sqlalchemy.engine.base.Engine ('Marketing',)
2020-12-04 12:14:43,371 INFO sqlalchemy.engine.base.Engine INSERT INTO employee (name) VALUES (?)
2020-12-04 12:14:43,373 INFO sqlalchemy.engine.base.Engine ('John',)
2020-12-04 12:14:43,374 INFO sqlalchemy.engine.base.Engine INSERT INTO employee (name) VALUES (?)
2020-12-04 12:14:43,374 INFO sqlalchemy.engine.base.Engine ('Graham',)
2020-12-04 12:14:43,375 INFO sqlalchemy.engine.base.Engine INSERT INTO 

In [8]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()

for x in session.query( Department, Employee).filter(Link.department_id == Department.id, 
    Link.employee_id == Employee.id).order_by(Link.department_id).all():
    print("Department: {} Name: {}".format(x.Department.name, x.Employee.name))

2020-12-04 12:14:43,390 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-04 12:14:43,391 INFO sqlalchemy.engine.base.Engine SELECT department.id AS department_id, department.name AS department_name, employee.id AS employee_id, employee.name AS employee_name 
FROM department, employee, link 
WHERE link.department_id = department.id AND link.employee_id = employee.id ORDER BY link.department_id
2020-12-04 12:14:43,392 INFO sqlalchemy.engine.base.Engine ()
Department: Accounts Name: John
Department: Accounts Name: Graham
Department: Sales Name: Graham
Department: Sales Name: Tony
Department: Marketing Name: John
Department: Marketing Name: Tony
